In [13]:
import cv2
import torch
ESRGAN_model_path = 'models/RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
device = torch.device('cuda')  

In [1]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coco128.yaml", epochs=3)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
path = model.export(format="onnx")  # export the model to ONNX format

/home/imonalc/anaconda3/envs/dice/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytic

In [5]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couc

In [7]:
type(results)

list

In [10]:
results[1]

IndexError: list index out of range

In [11]:
boxes = results[0].boxes

In [12]:
boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([[1.7191e+01, 2.2659e+02, 7.9526e+02, 7.5518e+02, 8.6290e-01, 5.0000e+00],
        [4.7968e+01, 3.9871e+02, 2.4510e+02, 9.0392e+02, 8.6118e-01, 0.0000e+00],
        [6.6920e+02, 3.7933e+02, 8.0975e+02, 8.7580e+02, 8.3914e-01, 0.0000e+00],
        [2.2123e+02, 4.0559e+02, 3.4478e+02, 8.5839e+02, 8.0248e-01, 0.0000e+00],
        [7.1296e-02, 2.5491e+02, 3.2210e+01, 3.2471e+02, 4.1606e-01, 1.1000e+01],
        [0.0000e+00, 5.5094e+02, 6.5356e+01, 8.7419e+02, 3.2356e-01, 0.0000e+00]], device='cuda:0')
cls: tensor([ 5.,  0.,  0.,  0., 11.,  0.], device='cuda:0')
conf: tensor([0.8629, 0.8612, 0.8391, 0.8025, 0.4161, 0.3236], device='cuda:0')
data: tensor([[1.7191e+01, 2.2659e+02, 7.9526e+02, 7.5518e+02, 8.6290e-01, 5.0000e+00],
        [4.7968e+01, 3.9871e+02, 2.4510e+02, 9.0392e+02, 8.6118e-01, 0.0000e+00],
        [6.6920e+02, 3.7933e+02, 8.0975e+02, 8.7580e+02, 8.3914e-01, 0.0000e+00],
        [2.2123e+02, 4.0559e+02,

In [24]:
def draw_boxes(img, boxes, classes, confidences):
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box[:4])
        class_id = int(classes[i])
        confidence = confidences[i]

        # バウンディングボックスとラベルの描画
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{class_id} {confidence:.2f}"
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 画像の読み込み
image = cv2.imread('bus.jpg')  # 適切な画像パスに置き換えてください

class_names = results[0].names

# resultsから必要なデータを取得
boxes = results[0].boxes.data.cpu().numpy()  # [x1, y1, x2, y2, conf, cls_id]
confidences = results[0].boxes.conf.cpu().numpy()
class_ids = results[0].boxes.cls.cpu().numpy()

# 検出されたオブジェクトのバウンディングボックスを描画
for idx, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box[:4])
    conf = confidences[idx]
    cls_id = int(class_ids[idx])
    label = class_names[cls_id]

    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# 結果を表示
cv2.imshow('YOLO Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
boxes = results[0].boxes.data.cpu().numpy()  # [x1, y1, x2, y2, conf, cls_id]
confidences = results[0].boxes.conf.cpu().numpy()
class_ids = results[0].boxes.cls.cpu().numpy()